In [1]:
import tensorflow.compat.v1 as tf
import numpy as np
import math
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
learning_rate   = 0.0008
batch_size      = 4096
sample_size     = 4096 # 4096 according to the paper
epochs          = 3000  # 850000 according to the paper
steps_per_epoch = int(sample_size/batch_size)

In [3]:
# BOB_LOSS_THRESH = 0.02  # Exit when Bob loss < 0.02 and Eve > 7.7 bits
# EVE_LOSS_THRESH = 7.7

In [4]:
# Input and output configuration.
TEXT_SIZE = 16
KEY_SIZE  = 16

In [5]:
# Training parameters.
ITERS_PER_ACTOR = 1
EVE_MULTIPLIER = 10  # Train Eve 2x for every step of Alice/Bob

In [6]:
# Set a random seed to help reproduce the output
seed = 7919
tf.set_random_seed(seed)
np.random.seed(seed)

# False if we want to train from scratch and true to contiune training a already trained model
restore_trained_model = False

In [7]:
def random_bools(sample_size, n):

  temp =  np.random.random_integers(0, high=1, size=[sample_size, n])
  temp = temp*2 - 1
  return temp.astype(np.float32)

In [8]:
# Function to create individual models for Alice, Bob and Eve.
def model(collection, message, key=None):
  
  if key is not None:
    # if there is a key then it's either Alice or Bob model trying to encrypt and decrypt resp
    # message and key is concatenated into single tensor and fed as an input
    combined_input = tf.concat(axis=1, values=[message, key])
  else:
    # if no key is present then it's Eve model trying to eavesdrop hence pass the message as input tensor 
    combined_input = message

  # collection arg is to denote the scope of model which is used to aggregate the 
  # tensor to make sure all tensor which needs to be trained are inside one scope
  with tf.variable_scope(collection):
    
    
    ## Fully connected layer of 16+16 = 32 neurons
    fc = tf.layers.dense(combined_input, TEXT_SIZE + KEY_SIZE, activation=tf.nn.relu)
    fc = tf.expand_dims(fc, 2)

    ## Convolution Layers - Sigmoid activation function

    # input: (32,1) -> output:(32,1) because filter is 1 which creates 1 channels
    conv1 = tf.layers.conv1d( fc,    filters=1, kernel_size=4, strides= 1, padding='SAME',  activation=tf.nn.leaky_relu)

    # input: (32,1) -> output:(16,2) because stride is 2 hence result is halved ( i.e 32/2 )
    # filter is 2 which creates 2 channels
    conv2 = tf.layers.conv1d( conv1, filters=2, kernel_size=2, strides= 2, padding='VALID', activation=tf.nn.leaky_relu)

    # input: (16,2) -> output:(16,1) 
    conv3 = tf.layers.conv1d( conv2, filters=1, kernel_size=1, strides= 1, padding='SAME',  activation=tf.nn.leaky_relu)

    # input: (16,1 ) -> output:(16,4) because filter is 4 which creates 4 channels
    conv4 = tf.layers.conv1d( conv3, filters=4, kernel_size=1, strides= 1, padding='SAME',  activation=tf.nn.sigmoid)
    
    # input: (16,4) -> output:(16,1) because filter is 1 which creates 1 channels
    conv5 = tf.layers.conv1d( conv4, filters=1, kernel_size=1, strides= 1, padding='SAME',  activation=tf.nn.relu)


    ## Convolution Layers - Tanh activation function

    # # input: (16,1) -> output:(16,1) because filter is 1 which creates 1 channel
    conv6 = tf.layers.conv1d( conv5, filters=1, kernel_size=1, strides=1, padding='SAME', activation=tf.nn.tanh)

    # Opposite of expand_dims function, here (16,1) tensor is converted to tensor of (16)
    out = tf.squeeze(conv6, 2)

  return out


In [9]:
#alice settings
Alice_input_message  = tf.placeholder(tf.float32, shape=(batch_size, TEXT_SIZE), name='Alice_input_message')
Alice_input_key      = tf.placeholder(tf.float32, shape=(batch_size, KEY_SIZE), name='Alice_input_key')

In [10]:
#Defined models
Alice_out_cipher = model('Alice', Alice_input_message, Alice_input_key)
Bob_out_message  = model('Bob', Alice_out_cipher, Alice_input_key)
Eve_out_message  = model('Eve', Alice_out_cipher)

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.


In [11]:
## Eves LOSS
Eves_loss = (1/batch_size)*tf.reduce_sum( tf.abs( Eve_out_message - Alice_input_message ))

In [12]:
## ALICE AND BOB LOSS
Bob_loss = (1/batch_size)*tf.reduce_sum( tf.abs( Bob_out_message  - Alice_input_message ))
Eve_evadropping_loss = tf.reduce_sum( tf.square(float(TEXT_SIZE) / 2.0 - Eves_loss) / ((TEXT_SIZE / 2)**2) )

In [13]:
Alice_bob_loss = Bob_loss + Eve_evadropping_loss

In [14]:
# Get tensors to train
Alice_vars =  tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope='Alice') 
Bob_vars   =  tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope='Bob') 
Eve_vars   =  tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='Eve') 

In [15]:
Eve_opt  = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.9, epsilon=1e-08).minimize(Eves_loss, var_list=[Eve_vars])
bob_opt  = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.9, epsilon=1e-08).minimize(Alice_bob_loss, var_list=[Alice_vars + Bob_vars])

In [16]:
sess = tf.Session() 
init = tf.global_variables_initializer()
sess.run(init)


In [17]:
alice_saver = tf.train.Saver(Alice_vars)
bob_saver   = tf.train.Saver(Bob_vars)
eve_saver   = tf.train.Saver(Eve_vars)

In [18]:
if restore_trained_model:
  alice_saver.restore(sess, "alice_weights.ckpt")
  bob_saver.restore(sess, "bob_weights.ckpt")
  eve_saver.restore(sess, "eve_weights.ckpt")

In [19]:
# DATASET 
messages = random_bools(sample_size, TEXT_SIZE)
keys     = random_bools(sample_size, KEY_SIZE)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
epochs_list = []
alice_bob_loss = []
eve_evs_loss = []
eve_loss = []

In [ ]:
# Training begins
for i in range(epochs):

  for j in range(steps_per_epoch):

    # get batch dataset to train
    batch_messages = messages[j*batch_size: (j+1)*batch_size]
    batch_keys     = keys[j*batch_size: (j+1)*batch_size]

    # Train Alice and Bob
    for _ in range(ITERS_PER_ACTOR):
      temp = sess.run([bob_opt, Bob_loss, Eve_evadropping_loss, Bob_out_message],feed_dict={Alice_input_message:batch_messages , Alice_input_key:batch_keys })
      
      temp_alice_bob_loss = temp[1]
      temp_eve_evs_loss   = temp[2]
      temp_bob_msg        = temp[3]

    # train Eve
    for _ in range(ITERS_PER_ACTOR*EVE_MULTIPLIER):
      temp = sess.run([Eve_opt, Eves_loss, Eve_out_message], feed_dict={Alice_input_message:batch_messages , Alice_input_key:batch_keys })

      temp_eve_loss = temp[1]
      temp_eve_msg  = temp[2]

  # save after every 500 epochs
  if i%500 == 0 and i!=0:
    alice_saver.save(sess, "alice_weights.ckpt")
    bob_saver.save(sess, "bob_weights.ckpt")
    eve_saver.save(sess, "eve_weights.ckpt")


  # output bit error and loss after every 100 epochs
  if i%50 == 0:
    epochs_list.append(i)
    alice_bob_loss.append(temp_alice_bob_loss)
    eve_evs_loss.append(temp_eve_evs_loss)
    eve_loss.append(temp_eve_loss)
    print('  epochs: ', i, '  bob bit error: ', temp_alice_bob_loss,' + ', temp_eve_evs_loss,'   & eve bit error:', temp_eve_loss)

sess.close()

  epochs:  0   bob bit error:  15.943176  +  1.0019789    & eve bit error: 16.00439
  epochs:  50   bob bit error:  15.930668  +  0.9926027    & eve bit error: 15.969902
  epochs:  100   bob bit error:  15.889614  +  0.98764396    & eve bit error: 15.950154
  epochs:  150   bob bit error:  15.41131  +  0.984824    & eve bit error: 15.938914
  epochs:  200   bob bit error:  13.484682  +  0.98321533    & eve bit error: 15.932491
  epochs:  250   bob bit error:  11.370431  +  0.98224723    & eve bit error: 15.928616
  epochs:  300   bob bit error:  9.224936  +  0.9816292    & eve bit error: 15.92614


In [ ]:
import pickle
pickle_out = open("epochs_list2.pickle","wb")
pickle.dump(epochs_list, pickle_out)
pickle_out.close()
pickle_out = open("alice_bob_loss2.pickle","wb")
pickle.dump(alice_bob_loss, pickle_out)
pickle_out.close()
pickle_out = open("eve_evs_loss2.pickle","wb")
pickle.dump(eve_evs_loss, pickle_out)
pickle_out.close()
pickle_out = open("eve_loss2.pickle","wb")
pickle.dump(eve_loss, pickle_out)
pickle_out.close()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import pickle
# pickle_in = open("epochs_list2.pickle","rb")
# pickle.load(epochs_list, pickle_in)
# pickle_in.close()
# pickle_in = open("alice_bob_loss2.pickle","rb")
# pickle.load(alice_bob_loss, pickle_in)
# pickle_in.close()
# pickle_in = open("eve_evs_loss2.pickle","rb")
# pickle.load(eve_evs_loss, pickle_in)
# pickle_in.close()
# pickle_in = open("eve_loss2.pickle","rb")
# pickle.load(eve_loss, pickle_in)
# pickle_in.close()

In [ ]:
offset = 0
jump=1
import matplotlib.pyplot as plt
from matplotlib.collections import EventCollection

fig, ax = plt.subplots()
ax.plot(epochs_list[offset::jump], alice_bob_loss[offset::jump], label="alice_bob_loss")
#ax.plot(epochs_list, eve_evs_loss, label="eve_evs_loss")
ax.plot(epochs_list[offset::jump], eve_loss[offset::jump], label="eve_loss")
ax.legend()

plt.show()

In [ ]:
epochs_list[100:]

In [ ]:
a = ""